In [ ]:
# Install PuLP first
!pip install pulp

# Import required libraries
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, value
import pandas as pd

# === Parameters ===
a = 0.000781
b = 2
fuel_price_ECA = 920
fuel_price_NonECA = 590
eca_distance_total = 680      # nm
noneca_distance_total = 0  # nm
speed_options = [15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0]

# === Step 1: Calculate benchmark time window ===
v_ref = 17.0  # reference cruising speed
time_eca_ref = eca_distance_total / v_ref
time_noneca_ref = noneca_distance_total / v_ref
time_total_ref = time_eca_ref + time_noneca_ref
Tmin = 0.9 * time_total_ref
Tmax = 1.1 * time_total_ref

# === Step 2: Define MILP model ===
model = LpProblem("Voyage_Optimisation_With_Time_Window", LpMinimize)

# Binary speed selection variables
x_eca = {v: LpVariable(f"x_eca_{v}", cat=LpBinary) for v in speed_options}
x_noneca = {v: LpVariable(f"x_noneca_{v}", cat=LpBinary) for v in speed_options}

# Fuel cost and time for each speed
eca_costs = {v: a * v**b * eca_distance_total * fuel_price_ECA for v in speed_options}
noneca_costs = {v: a * v**b * noneca_distance_total * fuel_price_NonECA for v in speed_options}
eca_times = {v: eca_distance_total / v for v in speed_options}
noneca_times = {v: noneca_distance_total / v for v in speed_options}

# Objective: minimise total fuel cost
model += (
    lpSum(eca_costs[v] * x_eca[v] for v in speed_options) +
    lpSum(noneca_costs[v] * x_noneca[v] for v in speed_options)
)

# Constraints
model += lpSum(x_eca[v] for v in speed_options) == 1
model += lpSum(x_noneca[v] for v in speed_options) == 1
total_time = lpSum(eca_times[v] * x_eca[v] for v in speed_options) + lpSum(noneca_times[v] * x_noneca[v] for v in speed_options)
model += total_time >= Tmin
model += total_time <= Tmax

# === Step 3: Solve model and extract results ===
model.solve()
v_eca = next(v for v in speed_options if value(x_eca[v]) == 1)
v_noneca = next(v for v in speed_options if value(x_noneca[v]) == 1)
total_cost = value(model.objective)
total_time_val = value(total_time)

# === Step 4: Display result ===
df_result = pd.DataFrame([{
    "Optimal ECA Speed (knots)": v_eca,
    "Optimal Non-ECA Speed (knots)": v_noneca,
    "Total Voyage Time (hr)": round(total_time_val, 2),
    "Total Fuel Cost (USD)": round(total_cost, 2),
    "Time Window (hr)": f"[{round(Tmin, 2)} – {round(Tmax, 2)}]"
}])

df_result


,Optimal ECA Speed (knots),Optimal Non-ECA Speed (knots),Total Voyage Time (hr),Total Fuel Cost (USD),Time Window (hr)
0,16.0,21.0,42.5,125079.96,[36.0 – 44.0]
